# Enhanced Quantum Teleportation Broadcast

ในส่วนนี้เราจะทำ broadcast โดยใช้ quantum teleportation ที่ทำให้:
- node1_epr[0], node2_epr[0], node3_epr[0] entangle กัน
- node1[0], node2[0], node3[0] entangle กัน

In [1]:
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

## สร้าง Enhanced Quantum Circuit

In [2]:
# สร้าง quantum circuit สำหรับ enhanced teleportation broadcast
enhanced_circuit = QuantumCircuit()

# สร้าง quantum registers สำหรับ nodes
node1 = QuantumRegister(1, name="node1")
node2 = QuantumRegister(1, name="node2") 
node3 = QuantumRegister(1, name="node3")

# สร้าง quantum registers สำหรับ EPR pairs
node1_epr = QuantumRegister(1, name="node1_epr")
node2_epr = QuantumRegister(1, name="node2_epr")
node3_epr = QuantumRegister(1, name="node3_epr")

# สร้าง classical registers สำหรับ teleportation measurements
tele1_measure = ClassicalRegister(2, name="tele1_measure")  # สำหรับ teleportation ครั้งที่ 1
tele2_measure = ClassicalRegister(2, name="tele2_measure")  # สำหรับ teleportation ครั้งที่ 2
tele3_measure = ClassicalRegister(2, name="tele3_measure")  # สำหรับ teleportation ครั้งที่ 3

# Final measurement register
final_measure = ClassicalRegister(3, name="final_measure")

# เพิ่ม registers เข้าใน circuit
enhanced_circuit.add_register(node1, node1_epr)
enhanced_circuit.add_register(node2, node2_epr)
enhanced_circuit.add_register(node3, node3_epr)
enhanced_circuit.add_register(tele1_measure, tele2_measure, tele3_measure, final_measure)

print(f"Enhanced circuit initialized with {enhanced_circuit.num_qubits} qubits")

Enhanced circuit initialized with 6 qubits


## Initialize และสร้าง Entanglement Networks

In [3]:
# Initialize node1[0] ให้มีโอกาส |0⟩ ~85%
enhanced_circuit.rx(math.pi/4, node1[0])

# สร้าง entanglement network สำหรับ EPR qubits
# ทำให้ node1_epr[0], node2_epr[0], node3_epr[0] entangle กัน
enhanced_circuit.h(node1_epr[0])
enhanced_circuit.cx(node1_epr[0], node2_epr[0])
enhanced_circuit.cx(node1_epr[0], node3_epr[0])

enhanced_circuit.barrier()
print("EPR entanglement network created")

EPR entanglement network created


## Quantum Teleportation Process

In [4]:
# Step 1: Teleport จาก node1[0] ไป node1_epr[0]
# Bell measurement
enhanced_circuit.cx(node1[0], node1_epr[0])
enhanced_circuit.h(node1[0])

# Measure teleportation bits
enhanced_circuit.measure(node1[0], tele1_measure[0])
enhanced_circuit.measure(node1_epr[0], tele1_measure[1])

enhanced_circuit.barrier()

# Apply corrections to entangled EPR qubits
# เนื่องจาก node2_epr[0] และ node3_epr[0] entangle กับ node1_epr[0]
# การแก้ไขจะส่งผลไปยัง qubits ทั้งหมดในเครือข่าย
with enhanced_circuit.if_test((tele1_measure[1], 1)):
    enhanced_circuit.x(node2_epr[0])
    enhanced_circuit.x(node3_epr[0])

with enhanced_circuit.if_test((tele1_measure[0], 1)):
    enhanced_circuit.z(node2_epr[0])
    enhanced_circuit.z(node3_epr[0])

enhanced_circuit.barrier()
print("Step 1: Teleportation to EPR network completed")

Step 1: Teleportation to EPR network completed


## Teleport จาก EPR Network ไป Target Nodes

In [5]:
# Step 2: Teleport จาก node2_epr[0] ไป node2[0]
enhanced_circuit.cx(node2_epr[0], node2[0])
enhanced_circuit.h(node2_epr[0])
enhanced_circuit.measure(node2_epr[0], tele2_measure[0])

with enhanced_circuit.if_test((tele2_measure[0], 1)):
    enhanced_circuit.z(node2[0])

# Step 3: Teleport จาก node3_epr[0] ไป node3[0]
enhanced_circuit.cx(node3_epr[0], node3[0])
enhanced_circuit.h(node3_epr[0])
enhanced_circuit.measure(node3_epr[0], tele3_measure[0])

with enhanced_circuit.if_test((tele3_measure[0], 1)):
    enhanced_circuit.z(node3[0])

enhanced_circuit.barrier()
print("Final teleportations completed")

# ตอนนี้ node1[0], node2[0], node3[0] ควรจะ entangle กัน
# เพราะพวกมันได้รับ state เดียวกันผ่าน teleportation network

Final teleportations completed


## Final Measurements และผลลัพธ์

In [6]:
# Final measurements
enhanced_circuit.measure(node1[0], final_measure[0])
enhanced_circuit.measure(node2[0], final_measure[1])
enhanced_circuit.measure(node3[0], final_measure[2])

print(f"Enhanced circuit stats:")
print(f"- Depth: {enhanced_circuit.depth()}")
print(f"- Qubits: {enhanced_circuit.num_qubits}")
print(f"- Classical bits: {enhanced_circuit.num_clbits}")

# Run enhanced simulation
def run_enhanced_broadcast():
    sim = AerSimulator()
    transpiled_circuit = transpile(enhanced_circuit, sim)
    result = sim.run(transpiled_circuit, shots=1024).result()
    counts = result.get_counts()
    
    print("\nEnhanced Teleportation Broadcast Results:")
    print("\nTop measurement outcomes:")
    for bits, count in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"{bits} : {count}")
    
    # วิเคราะห์ final measurements (3 bits สุดท้าย)
    total = sum(counts.values())
    
    # นับจำนวนครั้งที่ nodes มี state เดียวกัน
    same_state_count = 0
    for bits_str, count in counts.items():
        final_bits = bits_str.split()[-1]  # 3 bits สุดท้าย
        if final_bits[0] == final_bits[1] == final_bits[2]:  # ทุก node มี state เดียวกัน
            same_state_count += count
    
    correlation = same_state_count / total
    print(f"\nNode Correlation Analysis:")
    print(f"Same state probability: {correlation:.4f}")
    print(f"Expected for perfect broadcast: ~1.0")
    print(f"Broadcast Quality: {'Excellent' if correlation > 0.9 else 'Good' if correlation > 0.8 else 'Needs improvement'}")
    
    return counts

enhanced_counts = run_enhanced_broadcast()

Enhanced circuit stats:
- Depth: 13
- Qubits: 6
- Classical bits: 9

Enhanced Teleportation Broadcast Results:

Top measurement outcomes:
001 01 00 11 : 63
000 01 01 10 : 62
001 00 01 01 : 62
000 00 00 10 : 58
001 01 01 11 : 58
001 00 00 01 : 58
001 00 01 11 : 56
000 01 00 10 : 55
001 01 01 01 : 55
000 00 01 10 : 54


IndexError: string index out of range

## ตรวจสอบ Entanglement Properties

In [ ]:
# สร้าง circuit เพื่อตรวจสอบ entanglement โดยไม่ทำ final measurement
verification_circuit = QuantumCircuit()

# Copy setup จาก enhanced_circuit แต่ไม่ทำ final measurement
v_node1 = QuantumRegister(1, name="v_node1")
v_node2 = QuantumRegister(1, name="v_node2") 
v_node3 = QuantumRegister(1, name="v_node3")
v_node1_epr = QuantumRegister(1, name="v_node1_epr")
v_node2_epr = QuantumRegister(1, name="v_node2_epr")
v_node3_epr = QuantumRegister(1, name="v_node3_epr")

v_tele1_measure = ClassicalRegister(2, name="v_tele1_measure")
v_tele2_measure = ClassicalRegister(2, name="v_tele2_measure")
v_tele3_measure = ClassicalRegister(2, name="v_tele3_measure")

verification_circuit.add_register(v_node1, v_node1_epr)
verification_circuit.add_register(v_node2, v_node2_epr)
verification_circuit.add_register(v_node3, v_node3_epr)
verification_circuit.add_register(v_tele1_measure, v_tele2_measure, v_tele3_measure)

# Initialize
verification_circuit.rx(math.pi/4, v_node1[0])

# สร้าง EPR entanglement network
verification_circuit.h(v_node1_epr[0])
verification_circuit.cx(v_node1_epr[0], v_node2_epr[0])
verification_circuit.cx(v_node1_epr[0], v_node3_epr[0])

verification_circuit.barrier()

# Teleportation process
verification_circuit.cx(v_node1[0], v_node1_epr[0])
verification_circuit.h(v_node1[0])
verification_circuit.measure(v_node1[0], v_tele1_measure[0])
verification_circuit.measure(v_node1_epr[0], v_tele1_measure[1])

verification_circuit.barrier()

# Apply corrections
with verification_circuit.if_test((v_tele1_measure[1], 1)):
    verification_circuit.x(v_node2_epr[0])
    verification_circuit.x(v_node3_epr[0])

with verification_circuit.if_test((v_tele1_measure[0], 1)):
    verification_circuit.z(v_node2_epr[0])
    verification_circuit.z(v_node3_epr[0])

verification_circuit.barrier()

# Teleport to final nodes
verification_circuit.cx(v_node2_epr[0], v_node2[0])
verification_circuit.h(v_node2_epr[0])
verification_circuit.measure(v_node2_epr[0], v_tele2_measure[0])

with verification_circuit.if_test((v_tele2_measure[0], 1)):
    verification_circuit.z(v_node2[0])

verification_circuit.cx(v_node3_epr[0], v_node3[0])
verification_circuit.h(v_node3_epr[0])
verification_circuit.measure(v_node3_epr[0], v_tele3_measure[0])

with verification_circuit.if_test((v_tele3_measure[0], 1)):
    verification_circuit.z(v_node3[0])

print("Verification circuit setup completed")

## ทดสอบ Entanglement โดยการวัดใน Different Bases

In [ ]:
# ทดสอบการวัดใน computational basis
test_circuit_z = verification_circuit.copy()
test_measure_z = ClassicalRegister(3, name="test_z")
test_circuit_z.add_register(test_measure_z)

test_circuit_z.measure(v_node1[0], test_measure_z[0])
test_circuit_z.measure(v_node2[0], test_measure_z[1])
test_circuit_z.measure(v_node3[0], test_measure_z[2])

# ทดสอบการวัดใน X basis
test_circuit_x = verification_circuit.copy()
test_measure_x = ClassicalRegister(3, name="test_x")
test_circuit_x.add_register(test_measure_x)

# Rotate to X basis before measurement
test_circuit_x.h(v_node1[0])
test_circuit_x.h(v_node2[0])
test_circuit_x.h(v_node3[0])

test_circuit_x.measure(v_node1[0], test_measure_x[0])
test_circuit_x.measure(v_node2[0], test_measure_x[1])
test_circuit_x.measure(v_node3[0], test_measure_x[2])

print("Test circuits for entanglement verification created")

## รัน Tests และวิเคราะห์ผลลัพธ์

In [ ]:
def analyze_entanglement(circuit, test_name):
    sim = AerSimulator()
    transpiled = transpile(circuit, sim)
    result = sim.run(transpiled, shots=1024).result()
    counts = result.get_counts()
    
    print(f"\n{test_name} Measurement Results:")
    
    # วิเคราะห์ correlation
    total = sum(counts.values())
    same_state_count = 0
    
    for bits_str, count in counts.items():
        # หา final measurement bits (3 bits สุดท้าย)
        final_bits = bits_str.split()[-1]
        if len(final_bits) >= 3:
            node_bits = final_bits[-3:]  # 3 bits สุดท้าย
            if node_bits[0] == node_bits[1] == node_bits[2]:
                same_state_count += count
    
    correlation = same_state_count / total
    print(f"Correlation coefficient: {correlation:.4f}")
    
    # แสดง top results
    print("Top measurement outcomes:")
    for bits, count in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  {bits} : {count} ({count/total:.3f})")
    
    return correlation

# ทดสอบใน Z basis
z_correlation = analyze_entanglement(test_circuit_z, "Z-basis")

# ทดสอบใน X basis  
x_correlation = analyze_entanglement(test_circuit_x, "X-basis")

print(f"\n=== Entanglement Analysis Summary ===")
print(f"Z-basis correlation: {z_correlation:.4f}")
print(f"X-basis correlation: {x_correlation:.4f}")
print(f"Average correlation: {(z_correlation + x_correlation)/2:.4f}")

if z_correlation > 0.85 and x_correlation > 0.85:
    print("✓ Strong entanglement detected across all nodes!")
elif z_correlation > 0.7 or x_correlation > 0.7:
    print("~ Moderate entanglement detected")
else:
    print("✗ Weak or no entanglement detected")

## สรุปผลการทดลอง

In [ ]:
print("=== Quantum Teleportation Broadcast Summary ===")
print(f"\n1. Circuit Complexity:")
print(f"   - Total qubits: {enhanced_circuit.num_qubits}")
print(f"   - Circuit depth: {enhanced_circuit.depth()}")
print(f"   - Classical bits: {enhanced_circuit.num_clbits}")

print(f"\n2. Entanglement Network:")
print(f"   - EPR qubits (node1_epr, node2_epr, node3_epr) are entangled")
print(f"   - Target qubits (node1, node2, node3) receive entangled states via teleportation")

print(f"\n3. Teleportation Process:")
print(f"   - Step 1: node1[0] → node1_epr[0] (via Bell measurement)")
print(f"   - Step 2: Corrections applied to entire EPR network")
print(f"   - Step 3: node2_epr[0] → node2[0], node3_epr[0] → node3[0]")

print(f"\n4. Key Advantages:")
print(f"   - No direct interaction between distant nodes")
print(f"   - Uses pre-established EPR pairs")
print(f"   - Maintains quantum correlations across the network")
print(f"   - Scalable to more nodes by extending EPR network")